# DEVELOPING THE MAZE GENERATOR

## PREPARING THE DEVELOPMENT ENVIRONMENT

Importing the libraries:

In [1]:
from numpy.random import shuffle

Defining the auxiliary class with the disjoint set data structure:

- **n** is the number of supernodes
- **v** represents the parents (initially each disjoint set has a single vertex)

> **Note**: The compression technique used in the find method of disjoint set data structures is aimed at optimizing the process of finding the root or representative element of a set. In this technique, called path compression, as the method traverses the tree-like structure of the disjoint sets to find the root element, it also updates the parent pointers along the path to directly point to the root. This means that subsequent calls to find for the same element will take shorter paths, improving the efficiency of the operation.

In [2]:
class UnionFind:
    def __init__(self, n):
        self.n = n
        self.v = list(range(n))

    def find(self, u):
        while u != self.v[u]:
            self.v[u] = self.v[self.v[u]]
            u = self.v[u]

        return u

    def union(self, u, v):
        root_u, root_v = self.find(u), self.find(v)

        if root_u == root_v:
            return False
        else:
            self.v[root_v] = root_u
            self.n -= 1

            return True

Creating an auxiliary function to print the maze:

In [3]:
def is_conn(graph, v, w):
    return v in graph[w]

def get_row(maze, r, c, n):
    v = n * r + c

    return ' ' if is_conn(maze, v, v+1) else '|'

def get_col(maze, r, c, n):
    v = n * r + c

    return '  ' if is_conn(maze, v, v+n) else '_ '

def print_maze(n, maze):
    for r in range(n):
        for c in range(n-1):
            print('0', end=get_row(maze, r, c, n))

        print('0')

        if r != n-1:
            for c in range(n):
                print(end=get_col(maze, r, c, n))

            print()

## GENERATOR

Defining the class that implements a maze generator inspired by a randomized kruskal algorithm. The maze is represented using an adjacency list, as the resulting graph is expected to be sparse.

In [4]:
class MazeGenerator:
    def __init__(self, n=100):
        self.n = n
        self.shape = int(n**0.5)

        self.edges = self.generate_edges()

    def moves(self, v):
        moves = []

        if (v + 1) % self.shape:
            moves.append((v, v + 1))
        if v + self.shape < self.n:
            moves.append((v, v + self.shape))

        return moves

    def generate_edges(self):
        return [edge
                for v in range(self.n)
                for edge in self.moves(v)]

    def generate_maze(self):
        forest = UnionFind(self.n)
        maze   = [[] for _ in range(self.n)]

        shuffle(self.edges)

        for v, w in self.edges:
            if forest.union(v, w):
                maze[v].append(w)
                maze[w].append(v)

                if forest.n == 1:
                    return maze

Checking the possible valid edges:

In [5]:
generator = MazeGenerator()

edges = [[] for _ in range(generator.n)]
for v, w in generator.edges:
    edges[v].append(w)
    edges[w].append(v)

print_maze(generator.shape, edges)

0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0


Testing the generator and printing the results:

In [6]:
maze = generator.generate_maze()

print_maze(generator.shape, maze)

0|0 0|0|0 0 0|0 0 0
  _       _ _   _ _ 
0 0|0 0 0 0 0|0 0 0
    _ _     _     _ 
0|0 0 0 0|0|0 0|0 0
_ _ _   _ _   _   _ 
0 0 0|0|0|0 0|0|0 0
    _       _     _ 
0|0|0 0 0 0|0 0 0|0
_     _     _   _   
0 0 0|0|0|0|0 0 0 0
_   _   _       _   
0 0 0|0 0 0|0|0|0 0
_ _ _ _   _ _     _ 
0 0|0 0|0 0 0|0|0|0
      _   _         
0|0 0|0|0 0|0|0|0|0
_ _       _   _     
0 0 0 0 0|0 0|0 0 0


Checking the efficiency of the maze generator:

In [7]:
%%timeit

_ = generator.generate_maze()

55.8 µs ± 3.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
